In [1]:
from UtilityFunctions.run_query import run_query

In [2]:
query = 'SELECT ?s ?p ?o FROM <http://www.yelpkg.com/business> WHERE{?s ?p ?o } LIMIT 10'
# query = 'SELECT COUNT(?biz) FROM <http://www.yelpkg.com/business> WHERE {?biz rdfs:Class schema:LocalBusiness}'
# query = 'SELECT AVG(?rating) as ?averagerating WHERE {?biz schema:starRating ?rating}'
run_query(query=query, as_dataframe=False, do_print=True)
x=1

s: https://example.org/business_id/-a7VXX0-V9LgWMFrq90iNA; p: http://www.w3.org/2000/01/rdf-schema#Class; o: https://schema.org/LocalBusiness
s: https://example.org/business_id/0UqeZTDBdV0uY3wesbLvYQ; p: http://www.w3.org/2000/01/rdf-schema#Class; o: https://schema.org/LocalBusiness
s: https://example.org/business_id/1jx1sfgjgVg0nM6n3p0xWA; p: http://www.w3.org/2000/01/rdf-schema#Class; o: https://schema.org/LocalBusiness
s: https://example.org/business_id/2MAQeAqmD8enCT2ZYqUgIQ; p: http://www.w3.org/2000/01/rdf-schema#Class; o: https://schema.org/LocalBusiness
s: https://example.org/business_id/2O2K6SXPWv56amqxCECd4w; p: http://www.w3.org/2000/01/rdf-schema#Class; o: https://schema.org/LocalBusiness
s: https://example.org/business_id/5Z8iBpJMmOMz6G_7oVnzRA; p: http://www.w3.org/2000/01/rdf-schema#Class; o: https://schema.org/LocalBusiness
s: https://example.org/business_id/7omkeqEv-kKMIn9kmOR6Lw; p: http://www.w3.org/2000/01/rdf-schema#Class; o: https://schema.org/LocalBusiness
s: htt